# BDA Analyseprojekt 
Ilma Lupic 217098; Celina Wegner 216841; Malaika Hatos 216839

In [2]:
# Bibliotheken
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import math
import seaborn as sns

In [3]:
df = pd.read_csv("data/games.csv", index_col=False)
df.dtypes

C:\Users\malai\AppData\Local\Temp\ipykernel_23104\206405573.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv("data/games.csv", index_col=False)


AppID                           int64
Name                           object
Release date                   object
Estimated owners               object
Peak CCU                        int64
Required age                    int64
Price                         float64
DiscountDLC count               int64
About the game                  int64
Supported languages            object
Full audio languages           object
Reviews                        object
Header image                   object
Website                        object
Support url                    object
Support email                  object
Windows                        object
Mac                              bool
Linux                            bool
Metacritic score                 bool
Metacritic url                  int64
User score                     object
Positive                        int64
Negative                        int64
Score rank                      int64
Achievements                  float64
Recommendati

In [4]:
df.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DiscountDLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,0,Galactic Bowling is an exaggerated and stylize...,...,0,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,0,THE LAW!! Looks to be a showdown atop a train....,...,0,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,0,Jolt Project: The army now has a new robotics ...,...,0,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,...,0,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,0,ABOUT THE GAME Play as a hacker who has arrang...,...,0,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...


In [5]:
df.isnull().sum()

AppID                              0
Name                               6
Release date                       0
Estimated owners                   0
Peak CCU                           0
Required age                       0
Price                              0
DiscountDLC count                  0
About the game                     0
Supported languages             6483
Full audio languages               0
Reviews                            0
Header image                  100828
Website                            0
Support url                    64994
Support email                  60693
Windows                        19025
Mac                                0
Linux                              0
Metacritic score                   0
Metacritic url                     0
User score                    107447
Positive                           0
Negative                           0
Score rank                         0
Achievements                  111408
Recommendations                    0
N

In [6]:
df = df.drop(columns=["Header image"])
df = df.drop(columns=["Screenshots"])
df = df.drop(columns=["Support email"])
df = df.drop(columns=["Movies"])
df = df.drop(columns=["Support url"])

In [7]:
df = df.drop(columns=["User score"])
df = df.drop(columns=["Windows"])

In [8]:
df.isnull().sum()

AppID                              0
Name                               6
Release date                       0
Estimated owners                   0
Peak CCU                           0
Required age                       0
Price                              0
DiscountDLC count                  0
About the game                     0
Supported languages             6483
Full audio languages               0
Reviews                            0
Website                            0
Mac                                0
Linux                              0
Metacritic score                   0
Metacritic url                     0
Positive                           0
Negative                           0
Score rank                         0
Achievements                  111408
Recommendations                    0
Notes                              0
Average playtime forever       93003
Average playtime two weeks         0
Median playtime forever            0
Median playtime two weeks          0
D

In [9]:
df = df.drop(columns=["Achievements"])

In [10]:
df.isnull().sum()

AppID                             0
Name                              6
Release date                      0
Estimated owners                  0
Peak CCU                          0
Required age                      0
Price                             0
DiscountDLC count                 0
About the game                    0
Supported languages            6483
Full audio languages              0
Reviews                           0
Website                           0
Mac                               0
Linux                             0
Metacritic score                  0
Metacritic url                    0
Positive                          0
Negative                          0
Score rank                        0
Recommendations                   0
Notes                             0
Average playtime forever      93003
Average playtime two weeks        0
Median playtime forever           0
Median playtime two weeks         0
Developers                        0
Publishers                  

In [11]:
df = df.drop(columns=["Supported languages"])
df = df.drop(columns=["Categories"])
df = df.drop(columns=["Average playtime forever"])

In [12]:
df.isnull().sum()

AppID                            0
Name                             6
Release date                     0
Estimated owners                 0
Peak CCU                         0
Required age                     0
Price                            0
DiscountDLC count                0
About the game                   0
Full audio languages             0
Reviews                          0
Website                          0
Mac                              0
Linux                            0
Metacritic score                 0
Metacritic url                   0
Positive                         0
Negative                         0
Score rank                       0
Recommendations                  0
Notes                            0
Average playtime two weeks       0
Median playtime forever          0
Median playtime two weeks        0
Developers                       0
Publishers                    6475
Genres                        7566
Tags                          6440
dtype: int64

In [13]:
df = df.drop(columns=["Mac"])
df = df.drop(columns=["Linux"])
df = df.drop(columns=["Metacritic url"])
df = df.drop(columns=["Notes"]) ## = Average playtime forever?
df = df.drop(columns=["Developers"])

In [14]:
df.isnull().sum()

AppID                            0
Name                             6
Release date                     0
Estimated owners                 0
Peak CCU                         0
Required age                     0
Price                            0
DiscountDLC count                0
About the game                   0
Full audio languages             0
Reviews                          0
Website                          0
Metacritic score                 0
Positive                         0
Negative                         0
Score rank                       0
Recommendations                  0
Average playtime two weeks       0
Median playtime forever          0
Median playtime two weeks        0
Publishers                    6475
Genres                        7566
Tags                          6440
dtype: int64

In [15]:
#Prüfung ob Duplicate vorhanden sind
df.duplicated().sum()


np.int64(0)

In [16]:


pd.set_option('display.max_colwidth', None)  # Ganze Spalteninhalte anzeigen lassen


print(df)

          AppID                     Name  Release date Estimated owners  \
0         20200         Galactic Bowling  Oct 21, 2008        0 - 20000   
1        655370             Train Bandit  Oct 12, 2017        0 - 20000   
2       1732930             Jolt Project  Nov 17, 2021        0 - 20000   
3       1355720                 Henosis™  Jul 23, 2020        0 - 20000   
4       1139950    Two Weeks in Painland   Feb 3, 2020        0 - 20000   
...         ...                      ...           ...              ...   
111447  3600970          Paragon Of Time  Apr 10, 2025        0 - 20000   
111448  3543710  A Few Days With : Hazel  Apr 11, 2025        0 - 20000   
111449  3265370                 MosGhost   Apr 1, 2025        0 - 20000   
111450  3423620               AccuBow VR  Mar 11, 2025            0 - 0   
111451  3183790   Defense Of Fort Burton  Apr 10, 2025        0 - 20000   

        Peak CCU  Required age  Price  DiscountDLC count  About the game  \
0              0       

In [17]:
df[df.isnull().any(axis=1)]


,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DiscountDLC count,About the game,Full audio languages,...,Positive,Negative,Score rank,Recommendations,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Publishers,Genres,Tags
31,1727670,Home Office Tasker,"Sep 8, 2021",0 - 20000,0,0,0.99,0,0,"['English', 'Russian', 'German', 'Spanish - Spain', 'Simplified Chinese', 'Japanese', 'French']",...,0,11,4,0,0,0,0,lonch.me,NaN,Utilities
105,1943590,溪风谷之战 Playtest,"Mar 24, 2022",0 - 0,0,0,0.00,0,0,[],...,0,0,0,0,0,0,0,NaN,NaN,NaN
145,1302840,Kooring VR Coding Adventure,Aug 2020,0 - 20000,0,0,8.49,0,0,"['English', 'Simplified Chinese', 'Korean', 'Traditional Chinese']",...,0,8,0,0,0,0,0,VRANI inc.,NaN,"Adventure,Casual,Indie,Strategy,Education"
180,1966960,Burial Stone Playtest,"Apr 13, 2022",0 - 0,0,0,0.00,0,0,[],...,0,0,0,0,0,0,0,NaN,NaN,NaN
214,1688630,Emperial Knights Playtest,"Nov 14, 2021",0 - 0,0,0,0.00,0,0,[],...,0,0,0,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111400,3345250,Trailblazers: Into the March Playtest,"Apr 2, 2025",0 - 0,0,0,0.00,0,0,[],...,0,0,0,0,0,0,0,NaN,NaN,NaN
111405,3670540,Boiiing Boiiing Playtest,"Apr 18, 2025",0 - 0,0,0,0.00,0,0,[],...,0,0,0,0,0,0,0,NaN,NaN,NaN
111417,3609290,月之冕 Playtest,"Mar 25, 2025",0 - 0,0,0,0.00,0,0,[],...,0,0,0,5,0,0,0,NaN,NaN,NaN
111434,3654520,Delusional Playtest,"Apr 11, 2025",0 - 0,0,0,0.00,0,0,[],...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [18]:
df = df.dropna()

In [19]:
df.isnull().sum()

AppID                         0
Name                          0
Release date                  0
Estimated owners              0
Peak CCU                      0
Required age                  0
Price                         0
DiscountDLC count             0
About the game                0
Full audio languages          0
Reviews                       0
Website                       0
Metacritic score              0
Positive                      0
Negative                      0
Score rank                    0
Recommendations               0
Average playtime two weeks    0
Median playtime forever       0
Median playtime two weeks     0
Publishers                    0
Genres                        0
Tags                          0
dtype: int64